# Multiple Deep Learning Models for Key Press Detection

This notebook implements and compares multiple deep learning architectures for key press detection from video sequences using PyTorch:

## Available Models:
1. **CNN Model**: Pure convolutional neural network for single frame classification
2. **LSTM Model**: Sequential model using LSTM for temporal pattern recognition
3. **CNN+LSTM Model**: Combined architecture with CNN feature extraction + LSTM temporal modeling
4. **ResNet Model**: Residual network for robust feature extraction
5. **Transformer Model**: Attention-based model for sequence modeling

## Dataset Structure
The training data comes from the video labeler application with the following format:
- Image sequences (64x64x3) of key regions
- Binary labels (0: not pressed, 1: pressed)
- Temporal ordering for sequence modeling

## Training Scenarios
- **Single Frame Classification**: CNN, ResNet models
- **Sequence Classification**: LSTM, CNN+LSTM, Transformer models
- **Comparison Study**: Performance analysis across all models

## Framework
- **PyTorch**: Deep learning framework
- **PyTorch Lightning**: Training framework for clean, scalable code
- **Weights & Biases**: Experiment tracking and visualization

In [ ]:
# Import Required Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.models import resnet18, resnet34

import numpy as np
import cv2
import json
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Optional: PyTorch Lightning for clean training code
try:
    import pytorch_lightning as pl
    from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
    from pytorch_lightning.loggers import WandbLogger
    LIGHTNING_AVAILABLE = True
    print("✓ PyTorch Lightning available")
except ImportError:
    LIGHTNING_AVAILABLE = False
    print("⚠ PyTorch Lightning not available - using basic PyTorch training")

# Optional: Weights & Biases for experiment tracking
try:
    import wandb
    WANDB_AVAILABLE = True
    print("✓ Weights & Biases available")
except ImportError:
    WANDB_AVAILABLE = False
    print("⚠ Weights & Biases not available - using local logging")

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("Libraries imported successfully!")

In [ ]:
# Load and Preprocess Training Data
class KeypressDataset(Dataset):
    """PyTorch Dataset for keypress detection."""
    
    def __init__(self, images, labels, transform=None, sequence_length=None):
        self.images = torch.FloatTensor(images)
        
        # Check if labels are already one-hot encoded
        if len(labels.shape) > 1 and labels.shape[1] > 1:
            # Already one-hot encoded - use as is
            self.labels = torch.FloatTensor(labels)
            print(f"Using one-hot encoded labels of shape {self.labels.shape}")
        else:
            # Convert to LongTensor for class indices
            self.labels = torch.LongTensor(labels)
            print(f"Using class index labels of shape {self.labels.shape}")
        
        self.transform = transform
        self.sequence_length = sequence_length
        
        # Create sequences if sequence_length is provided
        if sequence_length is not None:
            self.create_sequences()
    
    def create_sequences(self):
        """Create sequences for temporal models."""
        sequences = []
        seq_labels = []
        
        for i in range(len(self.images) - self.sequence_length + 1):
            seq = self.images[i:i + self.sequence_length]
            label = self.labels[i + self.sequence_length - 1]  # Use last frame's label
            sequences.append(seq)
            seq_labels.append(label)
        
        self.images = torch.stack(sequences)
        self.labels = torch.stack(seq_labels)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            if self.sequence_length is not None:
                # Apply transform to each frame in sequence
                transformed_seq = []
                for frame in image:
                    transformed_seq.append(self.transform(frame))
                image = torch.stack(transformed_seq)
            else:
                image = self.transform(image)
        
        return image, label

def load_training_data(data_dir="labeled_data"):
    """Load training data from JSON files exported by video labeler."""
    
    # Find all training data files
    json_files = glob.glob(os.path.join(data_dir, "training_data_*.json"))
    
    if not json_files:
        raise FileNotFoundError(f"No training data found in {data_dir}")
    
    all_images = []
    all_labels = []
    all_frame_indices = []
    
    print(f"Found {len(json_files)} training data files:")
    
    for json_file in json_files:
        print(f"Loading: {json_file}")
        
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        # Extract sequence data
        sequence_data = data['sequence_data']
        
        for item in sequence_data:
            # Convert image list back to numpy array
            image = np.array(item['image'], dtype=np.float32)
            
            # Ensure image is in correct format (64, 64, 3)
            if image.shape != (64, 64, 3):
                print(f"Warning: Invalid image shape {image.shape}, skipping...")
                continue
            
            # Convert to PyTorch format (C, H, W)
            image = np.transpose(image, (2, 0, 1))
            all_images.append(image)
            all_labels.append(item['label'])
            all_frame_indices.append(item['frame_idx'])
        
        print(f"  - Loaded {len(sequence_data)} samples")
    
    # Convert to numpy arrays
    X = np.array(all_images)
    y = np.array(all_labels)
    frame_indices = np.array(all_frame_indices)
    
    print(f"\nTotal dataset:")
    print(f"  - Images shape: {X.shape}")
    print(f"  - Labels shape: {y.shape}")
    print(f"  - Label distribution: {np.bincount(y)}")
    
    return X, y, frame_indices

def create_data_transforms():
    """Create data transforms for training and validation."""
    
    # Training transforms (with augmentation)
    train_transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(p=0.1),  # Low probability for text
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
        transforms.RandomErasing(p=0.1, scale=(0.02, 0.1)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Validation transforms (no augmentation)
    val_transform = transforms.Compose([
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

# Load the data
try:
    X, y, frame_indices = load_training_data()
    print("Data loaded successfully!")
    
    # Create transforms
    train_transform, val_transform = create_data_transforms()
    print("Data transforms created!")
    
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please make sure you have exported training data from the video labeler.")

In [ ]:
# Data Exploration and Visualization
def visualize_dataset(X, y, frame_indices, num_samples=8):
    """Visualize sample images and analyze dataset."""
    
    if len(X) == 0:
        print("No data to visualize")
        return
    
    # Plot sample images
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('Sample Key Region Images', fontsize=16)
    
    # Show samples from each class
    pressed_indices = np.where(y == 1)[0]
    not_pressed_indices = np.where(y == 0)[0]
    
    for i in range(4):
        # Not pressed samples
        if i < len(not_pressed_indices):
            idx = not_pressed_indices[i]
            # Ensure image is in proper format for matplotlib
            img = X[idx].copy()
            # Convert from channels-first (3, 64, 64) to channels-last (64, 64, 3)
            img = np.transpose(img, (1, 2, 0))
            if img.max() <= 1.0:  # If normalized to [0,1]
                img = (img * 255).astype(np.uint8)
            elif img.min() < 0:  # If normalized to [-1,1]
                img = ((img + 1) * 127.5).astype(np.uint8)
            axes[0, i].imshow(img)
            axes[0, i].set_title(f'Not Pressed (Frame {frame_indices[idx]})')
            axes[0, i].axis('off')
        
        # Pressed samples
        if i < len(pressed_indices):
            idx = pressed_indices[i]
            # Ensure image is in proper format for matplotlib
            img = X[idx].copy()
            # Convert from channels-first (3, 64, 64) to channels-last (64, 64, 3)
            img = np.transpose(img, (1, 2, 0))
            if img.max() <= 1.0:  # If normalized to [0,1]
                img = (img * 255).astype(np.uint8)
            elif img.min() < 0:  # If normalized to [-1,1]
                img = ((img + 1) * 127.5).astype(np.uint8)
            axes[1, i].imshow(img)
            axes[1, i].set_title(f'Pressed (Frame {frame_indices[idx]})')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Label distribution
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    labels_count = np.bincount(y)
    plt.bar(['Not Pressed', 'Pressed'], labels_count, 
            color=['lightcoral', 'lightgreen'])
    plt.title('Label Distribution')
    plt.ylabel('Count')
    
    # Add percentage labels
    total = len(y)
    for i, count in enumerate(labels_count):
        plt.text(i, count + total*0.01, f'{count}\n({count/total:.1%})', 
                ha='center', va='bottom')
    
    plt.subplot(1, 2, 2)
    plt.hist(frame_indices, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Frame Index Distribution')
    plt.xlabel('Frame Index')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
    
    # Dataset statistics
    print(f"Dataset Statistics:")
    print(f"  - Total samples: {len(X)}")
    print(f"  - Image shape: {X.shape[1:]}")
    print(f"  - Image value range: [{X.min():.3f}, {X.max():.3f}]")
    print(f"  - Not pressed samples: {labels_count[0]} ({labels_count[0]/total:.1%})")
    print(f"  - Pressed samples: {labels_count[1]} ({labels_count[1]/total:.1%})")

# Visualize the dataset
if 'X' in locals() and len(X) > 0:
    visualize_dataset(X, y, frame_indices)
else:
    print("No data available for visualization")

In [ ]:
# Data Augmentation and Preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_data_augmentation():
    """Create data augmentation pipeline."""
    
    # Define augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=10,          # Random rotation
        width_shift_range=0.1,      # Random horizontal shift
        height_shift_range=0.1,     # Random vertical shift
        brightness_range=[0.8, 1.2], # Random brightness
        zoom_range=0.1,             # Random zoom
        horizontal_flip=False,       # No horizontal flip (text orientation)
        fill_mode='nearest',        # Fill mode for transformations
        rescale=None               # Don't rescale (already normalized)
    )
    
    return datagen

def augment_dataset(X, y, augment_factor=2):
    """Augment dataset to increase sample size."""
    
    if len(X) == 0:
        return X, y
    
    print(f"Original dataset size: {len(X)}")
    
    # Separate classes
    pressed_indices = np.where(y == 1)[0]
    not_pressed_indices = np.where(y == 0)[0]
    
    # Balance classes by augmenting minority class
    if len(pressed_indices) < len(not_pressed_indices):
        minority_class = 1
        minority_indices = pressed_indices
        majority_indices = not_pressed_indices
    else:
        minority_class = 0
        minority_indices = not_pressed_indices
        majority_indices = pressed_indices
    
    print(f"Minority class: {minority_class} ({len(minority_indices)} samples)")
    print(f"Majority class: {1-minority_class} ({len(majority_indices)} samples)")
    
    # Create augmentation generator
    datagen = create_data_augmentation()
    
    # Augment minority class
    X_minority = X[minority_indices]
    y_minority = y[minority_indices]
    
    # Generate augmented samples
    augmented_X = []
    augmented_y = []
    
    target_size = len(majority_indices)
    samples_needed = target_size - len(minority_indices)
    
    if samples_needed > 0:
        samples_per_original = samples_needed // len(minority_indices) + 1
        
        for i, (img, label) in enumerate(zip(X_minority, y_minority)):
            # Add original sample
            augmented_X.append(img)
            augmented_y.append(label)
            
            # Convert from PyTorch format (C, H, W) to Keras format (H, W, C)
            # Make sure we transpose the correct dimensions
            if img.shape[0] == 3 and img.shape[1] == 64 and img.shape[2] == 64:
                img_keras = np.transpose(img, (1, 2, 0))
            else:
                # If the image is not in the expected format, print an error message
                print(f"Unexpected image shape: {img.shape}, skipping augmentation for this sample")
                continue
                
            # Generate augmented samples
            img_batch = np.expand_dims(img_keras, axis=0)
            aug_iter = datagen.flow(img_batch, batch_size=1, shuffle=False)
            
            for _ in range(samples_per_original):
                if len(augmented_X) >= target_size:
                    break
                aug_img_keras = next(aug_iter)[0]
                
                # Convert back to PyTorch format (C, H, W)
                aug_img = np.transpose(aug_img_keras, (2, 0, 1))
                
                augmented_X.append(aug_img)
                augmented_y.append(label)
    
    # Combine original majority class with augmented minority class
    X_balanced = np.concatenate([X[majority_indices], np.array(augmented_X[:samples_needed])])
    y_balanced = np.concatenate([y[majority_indices], np.array(augmented_y[:samples_needed])])
    
    # Shuffle the balanced dataset
    shuffle_indices = np.random.permutation(len(X_balanced))
    X_balanced = X_balanced[shuffle_indices]
    y_balanced = y_balanced[shuffle_indices]
    
    print(f"Balanced dataset size: {len(X_balanced)}")
    print(f"New class distribution: {np.bincount(y_balanced)}")
    
    return X_balanced, y_balanced

def preprocess_data(X, y, balance_classes=True):
    """Preprocess data for training."""
    
    if len(X) == 0:
        return X, y
    
    # Ensure data is in correct format
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int32)
    
    # Print shape information for debugging
    print(f"Input X shape: {X.shape}")
    print(f"Expected format: (N, 3, 64, 64) - channels first")
    
    # Clip values to [0, 1] range (should already be normalized)
    X = np.clip(X, 0.0, 1.0)
    
    # Balance classes if requested
    if balance_classes:
        X, y = augment_dataset(X, y)
    
    # Convert labels to categorical
    y_categorical = np.eye(2)[y]  # One-hot encoding without keras dependency
    
    print(f"Preprocessed dataset:")
    print(f"  - X shape: {X.shape}")
    print(f"  - y shape: {y_categorical.shape}")
    print(f"  - X range: [{X.min():.3f}, {X.max():.3f}]")
    
    return X, y_categorical

# Preprocess the data
if 'X' in locals() and len(X) > 0:
    X_processed, y_processed = preprocess_data(X, y)
    print("Data preprocessing completed!")
else:
    print("No data available for preprocessing")

In [ ]:
# Multiple Model Architectures
class CNNModel(nn.Module):
    """Pure CNN model for single frame classification."""
    
    def __init__(self, num_classes=2):
        super(CNNModel, self).__init__()
        
        # Feature extraction layers
        self.features = nn.Sequential(
            # First conv block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Second conv block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Third conv block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Global average pooling
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

class LSTMModel(nn.Module):
    """LSTM model for sequence classification."""
    
    def __init__(self, input_size=64*64*3, hidden_size=128, num_layers=2, num_classes=2):
        super(LSTMModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=0.3, bidirectional=True)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, 256),  # *2 for bidirectional
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        # Reshape for LSTM (batch, seq_len, features)
        batch_size, seq_len, c, h, w = x.shape
        x = x.view(batch_size, seq_len, -1)
        
        # LSTM forward pass
        lstm_out, _ = self.lstm(x)
        
        # Use last output
        x = lstm_out[:, -1, :]
        
        # Classifier
        x = self.classifier(x)
        return x

class CNNLSTMModel(nn.Module):
    """CNN+LSTM model combining spatial and temporal features."""
    
    def __init__(self, num_classes=2):
        super(CNNLSTMModel, self).__init__()
        
        # CNN feature extractor
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # LSTM for temporal modeling
        self.lstm = nn.LSTM(128, 64, num_layers=2, 
                           batch_first=True, dropout=0.3, bidirectional=True)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(64 * 2, 128),  # *2 for bidirectional
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        batch_size, seq_len, c, h, w = x.shape
        
        # Extract CNN features for each frame
        cnn_features = []
        for i in range(seq_len):
            frame_features = self.cnn(x[:, i, :, :, :])
            frame_features = frame_features.view(batch_size, -1)
            cnn_features.append(frame_features)
        
        # Stack features for LSTM
        cnn_features = torch.stack(cnn_features, dim=1)
        
        # LSTM forward pass
        lstm_out, _ = self.lstm(cnn_features)
        
        # Use last output
        x = lstm_out[:, -1, :]
        
        # Classifier
        x = self.classifier(x)
        return x

class ResNetModel(nn.Module):
    """ResNet model for robust feature extraction."""
    
    def __init__(self, num_classes=2, pretrained=True):
        super(ResNetModel, self).__init__()
        
        # Load pretrained ResNet
        self.backbone = resnet18(pretrained=pretrained)
        
        # Replace final layer
        self.backbone.fc = nn.Sequential(
            nn.Linear(self.backbone.fc.in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)

class TransformerModel(nn.Module):
    """Transformer model for sequence classification."""
    
    def __init__(self, num_classes=2, d_model=256, nhead=8, num_layers=4):
        super(TransformerModel, self).__init__()
        
        self.d_model = d_model
        
        # CNN feature extractor
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # Projection to d_model
        self.projection = nn.Linear(128, d_model)
        
        # Positional encoding
        self.pos_encoding = nn.Parameter(torch.randn(1000, d_model))
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4, 
            dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        batch_size, seq_len, c, h, w = x.shape
        
        # Extract features for each frame
        features = []
        for i in range(seq_len):
            frame_features = self.feature_extractor(x[:, i, :, :, :])
            frame_features = frame_features.view(batch_size, -1)
            features.append(frame_features)
        
        # Stack and project features
        features = torch.stack(features, dim=1)
        features = self.projection(features)
        
        # Add positional encoding
        features = features + self.pos_encoding[:seq_len, :].unsqueeze(0)
        
        # Transformer forward pass
        transformer_out = self.transformer(features)
        
        # Use last output
        x = transformer_out[:, -1, :]
        
        # Classifier
        x = self.classifier(x)
        return x

# Model factory function
def create_model(model_type, num_classes=2, **kwargs):
    """Create model based on type."""
    
    models = {
        'cnn': CNNModel,
        'lstm': LSTMModel,
        'cnn_lstm': CNNLSTMModel,
        'resnet': ResNetModel,
        'transformer': TransformerModel
    }
    
    if model_type not in models:
        raise ValueError(f"Unknown model type: {model_type}")
    
    model = models[model_type](num_classes=num_classes, **kwargs)
    return model

# Display model information
print("Available models:")
for model_type in ['cnn', 'lstm', 'cnn_lstm', 'resnet', 'transformer']:
    model = create_model(model_type)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"  {model_type.upper()}: {total_params:,} total params, {trainable_params:,} trainable")

In [ ]:
# Training Pipeline for Multiple Models
class ModelTrainer:
    """Unified training pipeline for all models."""
    
    def __init__(self, model, device='cuda', learning_rate=0.001):
        self.model = model.to(device)
        self.device = device
        self.learning_rate = learning_rate
        
        # Loss function and optimizer
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=7, verbose=True
        )
        
        # Training history
        self.history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': [],
            'learning_rate': []
        }
    
    def train_epoch(self, train_loader):
        """Train for one epoch."""
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(self.device), target.to(self.device)
            
            # Handle target format - check if one-hot encoded or class indices
            if len(target.shape) > 1 and target.shape[1] > 1:
                # One-hot encoded - get the class indices
                _, target_indices = torch.max(target, 1)
            else:
                # Already class indices
                target_indices = target
            
            # Zero gradients
            self.optimizer.zero_grad()
            
            # Forward pass
            output = self.model(data)
            loss = self.criterion(output, target_indices)
            
            # Backward pass
            loss.backward()
            self.optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target_indices.size(0)
            correct += (predicted == target_indices).sum().item()
            
            if batch_idx % 10 == 0:
                print(f'Batch {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}, '
                      f'Acc: {100.*correct/total:.2f}%')
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    def validate_epoch(self, val_loader):
        """Validate for one epoch."""
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(self.device), target.to(self.device)
                
                # Handle target format - check if one-hot encoded or class indices
                if len(target.shape) > 1 and target.shape[1] > 1:
                    # One-hot encoded - get the class indices
                    _, target_indices = torch.max(target, 1)
                else:
                    # Already class indices
                    target_indices = target
                
                output = self.model(data)
                loss = self.criterion(output, target_indices)
                
                running_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                total += target_indices.size(0)
                correct += (predicted == target_indices).sum().item()
        
        epoch_loss = running_loss / len(val_loader)
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    def train(self, train_loader, val_loader, epochs=50, early_stopping_patience=15):
        """Train the model."""
        best_val_loss = float('inf')
        patience_counter = 0
        
        print(f"Training for {epochs} epochs...")
        print(f"Model: {self.model.__class__.__name__}")
        print(f"Device: {self.device}")
        print(f"Learning rate: {self.learning_rate}")
        print("-" * 50)
        
        for epoch in range(epochs):
            print(f'\nEpoch {epoch+1}/{epochs}')
            print('-' * 20)
            
            # Training
            train_loss, train_acc = self.train_epoch(train_loader)
            
            # Validation
            val_loss, val_acc = self.validate_epoch(val_loader)
            
            # Learning rate scheduling
            self.scheduler.step(val_loss)
            current_lr = self.optimizer.param_groups[0]['lr']
            
            # Save history
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['learning_rate'].append(current_lr)
            
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
            print(f'Learning Rate: {current_lr:.6f}')
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Save best model
                torch.save(self.model.state_dict(), f'best_{self.model.__class__.__name__.lower()}.pth')
                print("✓ New best model saved!")
            else:
                patience_counter += 1
                print(f"Early stopping patience: {patience_counter}/{early_stopping_patience}")
                
                if patience_counter >= early_stopping_patience:
                    print("Early stopping triggered!")
                    break
        
        print(f"\nTraining completed!")
        print(f"Best validation loss: {best_val_loss:.4f}")
        
        # Load best model
        self.model.load_state_dict(torch.load(f'best_{self.model.__class__.__name__.lower()}.pth'))
        
        return self.history

def prepare_dataloaders(X, y, model_type, batch_size=32, val_split=0.2, test_split=0.2):
    """Prepare data loaders for training."""
    
    # Print input shapes and types for debugging
    print(f"Input data types and shapes:")
    print(f"  X type: {type(X)}, shape: {X.shape}")
    print(f"  y type: {type(y)}, shape: {y.shape}")
    
    # Check if y is one-hot encoded
    is_one_hot = len(y.shape) > 1 and y.shape[1] > 1
    print(f"  Labels are {'one-hot encoded' if is_one_hot else 'class indices'}")
    
    # If one-hot encoded, convert to class indices for stratification
    if is_one_hot:
        y_indices = np.argmax(y, axis=1)
        print(f"  Converting to class indices for stratification, shape: {y_indices.shape}")
    else:
        y_indices = y
    
    # Determine if we need sequences
    sequence_models = ['lstm', 'cnn_lstm', 'transformer']
    sequence_length = 10 if model_type in sequence_models else None
    
    # Split data
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=test_split, random_state=42, stratify=y_indices
    )
    
    # For stratification of the validation split
    if is_one_hot:
        y_temp_indices = np.argmax(y_temp, axis=1)
    else:
        y_temp_indices = y_temp
        
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_split/(1-test_split), 
        random_state=42, stratify=y_temp_indices
    )
    
    print(f"Data splits:")
    print(f"  Train: {len(X_train)} samples")
    print(f"  Validation: {len(X_val)} samples")
    print(f"  Test: {len(X_test)} samples")
    if sequence_length:
        print(f"  Sequence length: {sequence_length}")
    
    # Create transforms
    train_transform, val_transform = create_data_transforms()
    
    # Create datasets
    train_dataset = KeypressDataset(X_train, y_train, train_transform, sequence_length)
    val_dataset = KeypressDataset(X_val, y_val, val_transform, sequence_length)
    test_dataset = KeypressDataset(X_test, y_test, val_transform, sequence_length)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return train_loader, val_loader, test_loader

def plot_training_history(history, model_name):
    """Plot training history."""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    ax1.plot(history['train_loss'], label='Training Loss')
    ax1.plot(history['val_loss'], label='Validation Loss')
    ax1.set_title(f'{model_name} - Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy
    ax2.plot(history['train_acc'], label='Training Accuracy')
    ax2.plot(history['val_acc'], label='Validation Accuracy')
    ax2.set_title(f'{model_name} - Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)
    
    # Learning rate
    ax3.plot(history['learning_rate'])
    ax3.set_title(f'{model_name} - Learning Rate')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_yscale('log')
    ax3.grid(True)
    
    # Training progress
    ax4.plot(history['val_loss'], label='Validation Loss')
    ax4.plot(history['val_acc'], label='Validation Accuracy')
    ax4.set_title(f'{model_name} - Validation Metrics')
    ax4.set_xlabel('Epoch')
    ax4.legend()
    ax4.grid(True)
    
    plt.tight_layout()
    plt.show()

# Training configuration
MODELS_TO_TRAIN = ['cnn', 'resnet', 'lstm', 'cnn_lstm', 'transformer']
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

print("Training pipeline ready!")
print(f"Models to train: {MODELS_TO_TRAIN}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")

In [ ]:
# Train Multiple Models and Compare Results
def train_all_models(X, y, models_to_train=None):
    """Train all models and compare results."""
    
    if models_to_train is None:
        models_to_train = MODELS_TO_TRAIN
    
    results = {}
    
    for model_type in models_to_train:
        print(f"\n{'='*60}")
        print(f"Training {model_type.upper()} Model")
        print(f"{'='*60}")
        
        try:
            # Prepare data loaders
            train_loader, val_loader, test_loader = prepare_dataloaders(
                X, y, model_type, batch_size=BATCH_SIZE
            )
            
            # Create model
            model = create_model(model_type)
            
            # Create trainer
            trainer = ModelTrainer(model, device=device, learning_rate=LEARNING_RATE)
            
            # Train model
            history = trainer.train(train_loader, val_loader, epochs=EPOCHS)
            
            # Plot training history
            plot_training_history(history, model_type.upper())
            
            # Evaluate on test set
            test_loss, test_acc = trainer.validate_epoch(test_loader)
            
            # Store results
            results[model_type] = {
                'model': trainer.model,
                'history': history,
                'test_loss': test_loss,
                'test_acc': test_acc,
                'train_loader': train_loader,
                'val_loader': val_loader,
                'test_loader': test_loader
            }
            
            print(f"\n{model_type.upper()} Results:")
            print(f"  Test Loss: {test_loss:.4f}")
            print(f"  Test Accuracy: {test_acc:.2f}%")
            
        except Exception as e:
            print(f"Error training {model_type}: {e}")
            results[model_type] = {'error': str(e)}
        
        print(f"\n{model_type.upper()} training completed!")
    
    return results

def compare_models(results):
    """Compare results from multiple models."""
    
    # Filter out failed models
    successful_results = {k: v for k, v in results.items() if 'error' not in v}
    
    if not successful_results:
        print("No successful model training results to compare.")
        return
    
    print("\n" + "="*60)
    print("MODEL COMPARISON RESULTS")
    print("="*60)
    
    # Create comparison table
    comparison_data = []
    for model_type, result in successful_results.items():
        final_val_acc = result['history']['val_acc'][-1]
        best_val_acc = max(result['history']['val_acc'])
        final_val_loss = result['history']['val_loss'][-1]
        best_val_loss = min(result['history']['val_loss'])
        
        comparison_data.append({
            'Model': model_type.upper(),
            'Test Accuracy': f"{result['test_acc']:.2f}%",
            'Best Val Accuracy': f"{best_val_acc:.2f}%",
            'Final Val Loss': f"{final_val_loss:.4f}",
            'Best Val Loss': f"{best_val_loss:.4f}",
            'Training Epochs': len(result['history']['train_loss'])
        })
    
    # Display comparison table
    df = pd.DataFrame(comparison_data)
    print(df.to_string(index=False))
    
    # Plot comparison
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Test accuracy comparison
    models = list(successful_results.keys())
    test_accs = [successful_results[m]['test_acc'] for m in models]
    
    ax1.bar(models, test_accs, color=['skyblue', 'lightgreen', 'lightcoral', 'gold', 'lightpink'][:len(models)])
    ax1.set_title('Test Accuracy Comparison')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_ylim(0, 100)
    
    # Add value labels on bars
    for i, v in enumerate(test_accs):
        ax1.text(i, v + 1, f'{v:.2f}%', ha='center', va='bottom')
    
    # Validation accuracy over time
    for model_type in models:
        history = successful_results[model_type]['history']
        ax2.plot(history['val_acc'], label=model_type.upper())
    
    ax2.set_title('Validation Accuracy Over Time')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)
    
    # Validation loss over time
    for model_type in models:
        history = successful_results[model_type]['history']
        ax3.plot(history['val_loss'], label=model_type.upper())
    
    ax3.set_title('Validation Loss Over Time')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Loss')
    ax3.legend()
    ax3.grid(True)
    
    # Training time comparison (epochs to convergence)
    epochs_to_convergence = [len(successful_results[m]['history']['train_loss']) for m in models]
    
    ax4.bar(models, epochs_to_convergence, color=['skyblue', 'lightgreen', 'lightcoral', 'gold', 'lightpink'][:len(models)])
    ax4.set_title('Training Time Comparison')
    ax4.set_ylabel('Epochs to Convergence')
    
    # Add value labels on bars
    for i, v in enumerate(epochs_to_convergence):
        ax4.text(i, v + 0.5, f'{v}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Find best model
    best_model = max(successful_results.keys(), key=lambda x: successful_results[x]['test_acc'])
    print(f"\n🏆 Best performing model: {best_model.upper()}")
    print(f"   Test Accuracy: {successful_results[best_model]['test_acc']:.2f}%")
    
    return successful_results

# Execute training if data is available
if 'X' in locals() and 'y' in locals() and len(X) > 0:
    print("Starting multi-model training...")
    
    # Train all models
    training_results = train_all_models(X, y)
    
    # Compare results
    comparison_results = compare_models(training_results)
    
    print("\n✅ Multi-model training completed!")
    
else:
    print("❌ No data available for training")
    print("Please run the data loading cell first")

In [ ]:
# Detailed Model Evaluation and Analysis
def evaluate_model_detailed(model, test_loader, model_name):
    """Detailed evaluation of a single model."""
    
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            # First, let's print the shape of batch_y to understand its format
            print(f"Debug - batch_y shape: {batch_y.shape}, batch_y type: {type(batch_y)}")
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # Forward pass
            outputs = model(batch_x)
            
            # Get predicted class
            _, predicted = torch.max(outputs.data, 1)
            
            # Also get probabilities for AUC and other metrics
            probs = F.softmax(outputs, dim=1).cpu().numpy()  # Using softmax instead of sigmoid
            
            # Convert to numpy for metrics calculation
            preds = predicted.cpu().numpy()
            
            # Handle labels based on their format
            if len(batch_y.shape) > 1 and batch_y.shape[1] > 1:
                # One-hot encoded labels
                batch_labels = torch.max(batch_y, 1)[1].cpu().numpy()
            else:
                # Class indices
                batch_labels = batch_y.cpu().numpy()
            
            all_preds.extend(preds)
            all_labels.extend(batch_labels)
            
            # For binary classification, take probability of class 1
            if probs.shape[1] >= 2:
                all_probs.extend(probs[:, 1])  # Probability of class 1 (pressed)
            else:
                all_probs.extend(probs.flatten())
    
    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    # Print debug info
    print(f"Debug Info:")
    print(f"  - all_preds shape: {all_preds.shape}")
    print(f"  - all_labels shape: {all_labels.shape}")
    print(f"  - all_probs shape: {all_probs.shape}")
    if len(all_preds) > 0:
        print(f"  - preds distribution: {np.bincount(all_preds)}")
    if len(all_labels) > 0:
        print(f"  - labels distribution: {np.bincount(all_labels.astype(int))}")
    
    # Check for length mismatch and fix if needed
    min_len = min(len(all_preds), len(all_labels), len(all_probs))
    if len(all_preds) != len(all_labels) or len(all_preds) != len(all_probs):
        print(f"⚠️ Length mismatch detected: preds={len(all_preds)}, labels={len(all_labels)}, probs={len(all_probs)}")
        print(f"   Truncating to minimum length: {min_len}")
        all_preds = all_preds[:min_len]
        all_labels = all_labels[:min_len]
        all_probs = all_probs[:min_len]
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except Exception as e:
        print(f"⚠️ AUC calculation failed: {e}")
        auc = 0.0
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    print(f"\n{model_name} - Detailed Evaluation:")
    print(f"  Samples evaluated: {len(all_labels)}")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"  AUC:       {auc:.4f}")
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Press', 'Key Press'], 
                yticklabels=['No Press', 'Key Press'])
    plt.title(f'{model_name} - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs
    }

def predict_single_image(model, image_array, model_type='cnn'):
    """
    Predicts key press for a single image.
    
    Args:
        model: The trained PyTorch model
        image_array: Numpy array of shape (H, W, C) or (C, H, W)
        model_type: Type of model ('cnn', 'resnet', 'lstm', 'cnn_lstm', 'transformer')
                   
    Returns:
        prediction: Class prediction (0 or 1)
        probability: Probability of the positive class (key press)
        class_probs: Probabilities for all classes
    """
    model.eval()
    
    # Ensure image is a numpy array
    if not isinstance(image_array, np.ndarray):
        raise ValueError("Image must be a numpy array")
    
    # Handle different image formats
    # Check if image is in PyTorch format (C, H, W) or standard format (H, W, C)
    if image_array.shape[0] == 3 and len(image_array.shape) == 3:
        # Already in PyTorch format (C, H, W)
        image_tensor = torch.FloatTensor(image_array)
    elif len(image_array.shape) == 3 and image_array.shape[2] == 3:
        # Convert from (H, W, C) to (C, H, W)
        image_tensor = torch.FloatTensor(np.transpose(image_array, (2, 0, 1)))
    else:
        raise ValueError(f"Unsupported image shape: {image_array.shape}, expected (3, H, W) or (H, W, 3)")
    
    # Handle sequence models
    if model_type in ['lstm', 'cnn_lstm', 'transformer']:
        # For sequence models, add sequence dimension
        image_tensor = image_tensor.unsqueeze(0).unsqueeze(0)  # (1, 1, C, H, W)
    else:
        # For CNN models, add batch dimension
        image_tensor = image_tensor.unsqueeze(0)  # (1, C, H, W)
    
    # Move to device and ensure model is in eval mode
    image_tensor = image_tensor.to(device)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(image_tensor)
        
        # Get class probabilities
        if outputs.shape[1] > 1:
            # Multi-class output
            probs = F.softmax(outputs, dim=1).cpu().numpy()[0]
            prediction = np.argmax(probs)
            probability = probs[1] if len(probs) > 1 else probs[0]
        else:
            # Binary output
            probs = torch.sigmoid(outputs).cpu().numpy()[0]
            prediction = (probs > 0.5).astype(int)[0]
            probability = probs[0]
            probs = np.array([1 - probability, probability])  # Convert to [not_pressed, pressed]
    
    return prediction, probability, probs

def predict_image_sequence(model, image_sequence, model_type='cnn_lstm'):
    """Predict keypress for a sequence of images."""
    
    model.eval()
    
    # Ensure image sequence is the right format
    if isinstance(image_sequence, list):
        image_sequence = np.array(image_sequence)
    
    # Check sequence format
    if len(image_sequence.shape) != 4:  # (seq_len, C, H, W)
        raise ValueError(f"Expected sequence shape (seq_len, C, H, W), got {image_sequence.shape}")
    
    # Convert to tensor
    seq_tensor = torch.FloatTensor(image_sequence).unsqueeze(0)  # Add batch dim
    seq_tensor = seq_tensor.to(device)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(seq_tensor)
        
        # Get class probabilities
        if outputs.shape[1] > 1:
            # Multi-class output
            probs = F.softmax(outputs, dim=1).cpu().numpy()[0]
            prediction = np.argmax(probs)
            probability = probs[1] if len(probs) > 1 else probs[0]
        else:
            # Binary output
            probs = torch.sigmoid(outputs).cpu().numpy()[0]
            prediction = (probs > 0.5).astype(int)[0]
            probability = probs[0]
            probs = np.array([1 - probability, probability])  # Convert to [not_pressed, pressed]
    
    return prediction, probability, probs

def analyze_predictions(results, model_names):
    """Analyze predictions across all models."""
    
    print("\n" + "="*60)
    print("DETAILED PREDICTION ANALYSIS")
    print("="*60)
    
    # Create comprehensive metrics table
    metrics_data = []
    for model_name in model_names:
        if model_name in results and 'error' not in results[model_name]:
            eval_results = evaluate_model_detailed(
                results[model_name]['model'], 
                results[model_name]['test_loader'], 
                model_name.upper()
            )
            
            metrics_data.append({
                'Model': model_name.upper(),
                'Accuracy': f"{eval_results['accuracy']:.4f}",
                'Precision': f"{eval_results['precision']:.4f}",
                'Recall': f"{eval_results['recall']:.4f}",
                'F1-Score': f"{eval_results['f1']:.4f}",
                'AUC': f"{eval_results['auc']:.4f}"
            })
    
    # Display metrics table
    if metrics_data:
        df = pd.DataFrame(metrics_data)
        print("\n📊 Comprehensive Metrics Comparison:")
        print(df.to_string(index=False))
        
        # Plot metrics comparison
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
        
        for i, metric in enumerate(metrics):
            values = [float(row[metric]) for row in metrics_data]
            models = [row['Model'] for row in metrics_data]
            
            bars = axes[i].bar(models, values, color=['skyblue', 'lightgreen', 'lightcoral', 'gold', 'lightpink'][:len(models)])
            axes[i].set_title(f'{metric} Comparison')
            axes[i].set_ylabel(metric)
            axes[i].set_ylim(0, 1)
            
            # Add value labels
            for bar, value in zip(bars, values):
                axes[i].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01, 
                           f'{value:.3f}', ha='center', va='bottom')
        
        # Remove empty subplot
        axes[5].remove()
        
        plt.tight_layout()
        plt.show()

# Example of using predict_single_image
def demonstrate_single_image_prediction():
    """Demonstrate using predict_single_image with a sample image."""
    if 'training_results' in locals() and 'X_processed' in locals():
        # Get a sample image from the test data
        test_loader = next(iter(training_results.values()))['test_loader']
        batch_x, _ = next(iter(test_loader))
        sample_image = batch_x[0].numpy()  # First image in batch
        
        # Test with each model
        for model_name, result in training_results.items():
            if 'error' not in result:
                model = result['model']
                prediction, probability, class_probs = predict_single_image(
                    model, sample_image, model_type=model_name
                )
                
                print(f"{model_name.upper()} prediction:")
                print(f"  Class: {'Pressed' if prediction == 1 else 'Not Pressed'}")
                print(f"  Probability: {probability:.4f}")
                print(f"  Class probabilities: {class_probs}")
                print()

# Perform detailed evaluation if training results exist
if 'training_results' in locals():
    analyze_predictions(training_results, list(training_results.keys()))
else:
    print("❌ No training results available for detailed evaluation")
    print("Please run the training cell first")

In [ ]:
# Model Inference and Prediction Functions
def detect_model_type(model):
    """Automatically detect the model type based on architecture."""
    model_name = model.__class__.__name__.lower()
    
    if 'lstm' in model_name and 'cnn' in model_name:
        return 'cnn_lstm'
    elif 'lstm' in model_name:
        return 'lstm'
    elif 'cnn' in model_name or 'resnet' in model_name or 'transformer' in model_name:
        return 'cnn'
    else:
        # Default fallback
        return 'cnn'

def predict_single_image(model, image_array, model_type=None):
    """Predict keypress for a single image."""
    
    model.eval()
    
    # Auto-detect model type if not provided
    if model_type is None:
        model_type = detect_model_type(model)
    
    # Prepare image - ensure it matches training data format
    if len(image_array.shape) == 3:
        image_array = np.expand_dims(image_array, axis=0)
    
    # Convert to tensor and move to device
    image_tensor = torch.FloatTensor(image_array).to(device)
    
    # Handle different model types
    if model_type in ['lstm', 'cnn_lstm']:
        # For LSTM models, we need sequence dimension
        if len(image_tensor.shape) == 4:
            image_tensor = image_tensor.unsqueeze(1)  # Add sequence dimension
    
    try:
        with torch.no_grad():
            outputs = model(image_tensor)
            
            # Use the same approach as evaluate_model_detailed
            if len(outputs.shape) > 1 and outputs.shape[1] > 1:  # Multi-class output
                # Get predicted class using argmax
                _, predicted = torch.max(outputs.data, 1)
                # Get probabilities using softmax
                probs = torch.softmax(outputs, dim=1).cpu().numpy()
                prediction = predicted.cpu().item()
                probability = probs[0][1] if probs.shape[1] > 1 else probs[0][0]
            else:  # Single output (binary classification with sigmoid)
                # Use sigmoid for single output
                probability = torch.sigmoid(outputs).cpu().numpy()[0][0]
                prediction = int(probability > 0.5)
        
        return prediction, probability
    
    except Exception as e:
        print(f"❌ Error in predict_single_image: {e}")
        print(f"   Model type: {model_type}")
        print(f"   Input shape: {image_tensor.shape}")
        return 0, 0.0

def predict_single_image_robust(model, image_array, model_type=None, transforms=None):
    """Enhanced single image prediction with proper preprocessing."""
    
    model.eval()
    
    # Auto-detect model type if not provided
    if model_type is None:
        model_type = detect_model_type(model)
    
    # Apply transforms if provided (should match training transforms)
    if transforms is not None:
        # Convert numpy array to PIL Image if needed
        if isinstance(image_array, np.ndarray):
            if image_array.dtype != np.uint8:
                image_array = (image_array * 255).astype(np.uint8)
            from PIL import Image
            if len(image_array.shape) == 3:
                image_pil = Image.fromarray(image_array)
            else:
                image_pil = Image.fromarray(image_array[0])
            
            # Apply transforms
            image_tensor = transforms(image_pil).unsqueeze(0).to(device)
        else:
            image_tensor = transforms(image_array).unsqueeze(0).to(device)
    else:
        # Manual preprocessing
        if len(image_array.shape) == 3:
            image_array = np.expand_dims(image_array, axis=0)
        
        # Normalize to [0, 1] if not already
        if image_array.max() > 1.0:
            image_array = image_array.astype(np.float32) / 255.0
        
        # Convert to tensor
        image_tensor = torch.FloatTensor(image_array).to(device)
    
    # Handle different model types
    if model_type in ['lstm', 'cnn_lstm']:
        if len(image_tensor.shape) == 4:
            image_tensor = image_tensor.unsqueeze(1)
    
    try:
        with torch.no_grad():
            outputs = model(image_tensor)
            
            # Debug information
            print(f"Model type: {model_type}")
            print(f"Input shape: {image_tensor.shape}")
            print(f"Model output shape: {outputs.shape}")
            
            # Handle output based on shape
            if len(outputs.shape) > 1 and outputs.shape[1] > 1:  # Multi-class
                # Use softmax and argmax like in evaluate_model_detailed
                probs = torch.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)
                
                prediction = predicted.cpu().item()
                probability = probs[0][1].cpu().item() if probs.shape[1] > 1 else probs[0][0].cpu().item()
                
                print(f"Multi-class prediction: {prediction}, probability: {probability}")
            else:  # Single output
                # Use sigmoid
                probability = torch.sigmoid(outputs).cpu().item()
                prediction = int(probability > 0.5)
                
                print(f"Binary prediction: {prediction}, probability: {probability}")
        
        return prediction, probability
    
    except Exception as e:
        print(f"❌ Error in predict_single_image_robust: {e}")
        print(f"   Model type: {model_type}")
        print(f"   Input shape: {image_tensor.shape}")
        return 0, 0.0

def predict_image_sequence(model, image_sequence, model_type=None):
    """Predict keypress for a sequence of images (for LSTM models)."""
    
    model.eval()
    
    # Auto-detect model type if not provided
    if model_type is None:
        model_type = detect_model_type(model)
    
    # Prepare sequence
    if len(image_sequence.shape) == 4:
        image_sequence = np.expand_dims(image_sequence, axis=0)
    
    # Convert to tensor
    sequence_tensor = torch.FloatTensor(image_sequence).to(device)
    
    try:
        with torch.no_grad():
            outputs = model(sequence_tensor)
            
            # Use consistent output handling
            if len(outputs.shape) > 1 and outputs.shape[1] > 1:  # Multi-class
                probs = torch.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)
                prediction = predicted.cpu().item()
                probability = probs[0][1].cpu().item()
            else:  # Single output
                probability = torch.sigmoid(outputs).cpu().item()
                prediction = int(probability > 0.5)
        
        return prediction, probability
    
    except Exception as e:
        print(f"❌ Error in predict_image_sequence: {e}")
        return 0, 0.0

def batch_predict(model, images, model_type=None, batch_size=32):
    """Predict keypress for multiple images in batches."""
    
    model.eval()
    
    # Auto-detect model type if not provided
    if model_type is None:
        model_type = detect_model_type(model)
    
    predictions = []
    probabilities = []
    
    # Process in batches
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i+batch_size]
        
        # Convert to tensor
        batch_tensor = torch.FloatTensor(batch_images).to(device)
        
        # Handle different model types
        if model_type in ['lstm', 'cnn_lstm']:
            if len(batch_tensor.shape) == 4:
                batch_tensor = batch_tensor.unsqueeze(1)
        
        try:
            with torch.no_grad():
                outputs = model(batch_tensor)
                
                # Use consistent output handling
                if len(outputs.shape) > 1 and outputs.shape[1] > 1:  # Multi-class
                    probs = torch.softmax(outputs, dim=1).cpu().numpy()
                    batch_preds = np.argmax(probs, axis=1)
                    batch_probs = probs[:, 1] if probs.shape[1] > 1 else probs[:, 0]
                else:  # Single output
                    batch_probs = torch.sigmoid(outputs).cpu().numpy().flatten()
                    batch_preds = (batch_probs > 0.5).astype(int)
            
            predictions.extend(batch_preds)
            probabilities.extend(batch_probs)
            
        except Exception as e:
            print(f"❌ Error in batch {i//batch_size + 1}: {e}")
            # Fill with default values for this batch
            batch_size_actual = len(batch_images)
            predictions.extend([0] * batch_size_actual)
            probabilities.extend([0.0] * batch_size_actual)
    
    return predictions, probabilities

def test_single_image_prediction(model, test_loader, model_name, num_tests=5):
    """Test single image prediction against batch evaluation."""
    
    print(f"\n🧪 Testing single image prediction for {model_name}")
    print("="*50)
    
    model.eval()
    
    # Auto-detect model type
    model_type = detect_model_type(model)
    print(f"Detected model type: {model_type}")
    
    # Get some test samples
    batch_x, batch_y = next(iter(test_loader))
    
    for i in range(min(num_tests, len(batch_x))):
        try:
            # Get single image
            single_image = batch_x[i].cpu().numpy()
            true_label = batch_y[i].cpu().item() if len(batch_y[i].shape) == 0 else batch_y[i].cpu().numpy()
            
            # Predict using single image function
            pred_single, prob_single = predict_single_image(model, single_image, model_type)
            
            # Predict using batch (for comparison)
            with torch.no_grad():
                batch_tensor = batch_x[i:i+1].to(device)
                
                # Handle LSTM models properly
                if model_type in ['lstm', 'cnn_lstm'] and len(batch_tensor.shape) == 4:
                    batch_tensor = batch_tensor.unsqueeze(1)
                
                outputs = model(batch_tensor)
                
                if len(outputs.shape) > 1 and outputs.shape[1] > 1:
                    probs = torch.softmax(outputs, dim=1)
                    _, predicted = torch.max(outputs, 1)
                    pred_batch = predicted.cpu().item()
                    prob_batch = probs[0][1].cpu().item()
                else:
                    prob_batch = torch.sigmoid(outputs).cpu().item()
                    pred_batch = int(prob_batch > 0.5)
            
            # Compare results
            match = "✅" if pred_single == pred_batch else "❌"
            print(f"Sample {i+1}: True={true_label} | Single={pred_single}({prob_single:.3f}) | Batch={pred_batch}({prob_batch:.3f}) {match}")
            
        except Exception as e:
            print(f"❌ Error testing sample {i+1}: {e}")
            continue
    
    return True

def visualize_predictions(model, test_loader, model_name, num_samples=16):
    """Visualize model predictions on test samples."""
    
    model.eval()
    
    # Auto-detect model type
    model_type = detect_model_type(model)
    
    # Get a batch of test data
    batch_x, batch_y = next(iter(test_loader))
    batch_x = batch_x[:num_samples]
    batch_y = batch_y[:num_samples]
    
    # Make predictions
    try:
        with torch.no_grad():
            batch_x_gpu = batch_x.to(device)
            
            # Handle LSTM models properly
            if model_type in ['lstm', 'cnn_lstm'] and len(batch_x_gpu.shape) == 4:
                batch_x_gpu = batch_x_gpu.unsqueeze(1)
            
            outputs = model(batch_x_gpu)
            
            # Handle different output formats
            if len(outputs.shape) > 1 and outputs.shape[1] > 1:  # Multi-class
                probs = torch.softmax(outputs, dim=1).cpu().numpy()
                preds = np.argmax(probs, axis=1)
                probs_display = probs[:, 1] if probs.shape[1] > 1 else probs[:, 0]
            else:  # Single output
                probs_display = torch.sigmoid(outputs).cpu().numpy().flatten()
                preds = (probs_display > 0.5).astype(int)
        
        # Create visualization
        fig, axes = plt.subplots(4, 4, figsize=(16, 16))
        axes = axes.flatten()
        
        for i in range(num_samples):
            # Get image
            if len(batch_x[i].shape) == 4:  # For LSTM models
                img = batch_x[i][0].permute(1, 2, 0).numpy()
            else:  # For CNN models
                img = batch_x[i].permute(1, 2, 0).numpy()
            
            # Denormalize if needed
            if img.min() < 0:
                img = (img + 1) / 2
            
            # Plot
            axes[i].imshow(img)
            axes[i].axis('off')
            
            # Add prediction info
            true_label = "Press" if batch_y[i].item() == 1 else "No Press"
            pred_label = "Press" if preds[i] == 1 else "No Press"
            prob = probs_display[i]
            
            color = 'green' if preds[i] == batch_y[i].item() else 'red'
            axes[i].set_title(f'True: {true_label}\nPred: {pred_label}\nProb: {prob:.3f}', 
                             color=color, fontsize=10)
        
        plt.suptitle(f'{model_name} - Sample Predictions (Type: {model_type})', fontsize=16)
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Error in visualization: {e}")
        print(f"   Model type: {model_type}")
        print(f"   Input shape: {batch_x_gpu.shape}")

def test_inference_speed(model, test_loader, model_name):
    """Test inference speed of the model."""
    
    model.eval()
    
    # Auto-detect model type
    model_type = detect_model_type(model)
    
    try:
        # Warm up
        with torch.no_grad():
            batch_x, _ = next(iter(test_loader))
            batch_x = batch_x.to(device)
            
            # Handle LSTM models
            if model_type in ['lstm', 'cnn_lstm'] and len(batch_x.shape) == 4:
                batch_x = batch_x.unsqueeze(1)
            
            _ = model(batch_x)
        
        # Time inference
        import time
        
        start_time = time.time()
        total_samples = 0
        
        with torch.no_grad():
            for batch_x, _ in test_loader:
                batch_x = batch_x.to(device)
                
                # Handle LSTM models
                if model_type in ['lstm', 'cnn_lstm'] and len(batch_x.shape) == 4:
                    batch_x = batch_x.unsqueeze(1)
                
                _ = model(batch_x)
                total_samples += batch_x.size(0)
        
        end_time = time.time()
        
        total_time = end_time - start_time
        fps = total_samples / total_time
        
        print(f"\n{model_name} Inference Speed:")
        print(f"  Model type: {model_type}")
        print(f"  Total samples: {total_samples}")
        print(f"  Total time: {total_time:.2f} seconds")
        print(f"  FPS: {fps:.2f} frames/second")
        print(f"  Average time per frame: {1/fps*1000:.2f} ms")
        
        return fps
    
    except Exception as e:
        print(f"❌ Error in speed test: {e}")
        return 0

# Demo predictions if training results exist
if 'training_results' in locals():
    print("🔮 Running inference demonstrations...")
    
    # Test inference for each successful model
    for model_name, result in training_results.items():
        if 'error' not in result:
            print(f"\n{'='*40}")
            print(f"Testing {model_name.upper()} Model")
            print(f"{'='*40}")
            
            try:
                # Test single image prediction consistency
                test_single_image_prediction(result['model'], result['test_loader'], model_name.upper())
                
                # Visualize predictions
                visualize_predictions(result['model'], result['test_loader'], model_name.upper())
                
                # Test inference speed
                fps = test_inference_speed(result['model'], result['test_loader'], model_name.upper())
                
                print(f"\n✅ {model_name.upper()} inference testing completed!")
                
            except Exception as e:
                print(f"❌ Error testing {model_name.upper()}: {e}")
                continue
    
else:
    print("❌ No training results available for inference testing")
    print("Please run the training cell first")

In [ ]:
# Hyperparameter Optimization and Tuning
def optimize_hyperparameters(X, y, model_type='cnn', max_trials=10):
    """Optimize hyperparameters using random search."""
    
    print(f"🔍 Starting hyperparameter optimization for {model_type.upper()}")
    
    # Define hyperparameter search space
    param_space = {
        'learning_rate': [0.001, 0.0001, 0.00001],
        'batch_size': [16, 32, 64],
        'dropout_rate': [0.2, 0.3, 0.5],
        'hidden_size': [64, 128, 256] if model_type in ['lstm', 'cnn_lstm'] else [None],
        'num_layers': [1, 2, 3] if model_type in ['lstm', 'cnn_lstm'] else [None]
    }
    
    best_score = 0
    best_params = None
    trial_results = []
    
    for trial in range(max_trials):
        print(f"\n--- Trial {trial + 1}/{max_trials} ---")
        
        # Sample random hyperparameters
        params = {}
        for param_name, values in param_space.items():
            if values[0] is not None:  # Skip None values
                params[param_name] = random.choice(values)
        
        print(f"Testing parameters: {params}")
        
        try:
            # Prepare data with current batch size
            train_loader, val_loader, test_loader = prepare_dataloaders(
                X, y, model_type, batch_size=params.get('batch_size', 32)
            )
            
            # Create model with current parameters
            if model_type == 'cnn':
                model = CNNModel(dropout_rate=params.get('dropout_rate', 0.3))
            elif model_type == 'lstm':
                model = LSTMModel(
                    hidden_size=params.get('hidden_size', 128),
                    num_layers=params.get('num_layers', 2),
                    dropout_rate=params.get('dropout_rate', 0.3)
                )
            elif model_type == 'cnn_lstm':
                model = CNNLSTMModel(
                    lstm_hidden_size=params.get('hidden_size', 128),
                    lstm_num_layers=params.get('num_layers', 2),
                    dropout_rate=params.get('dropout_rate', 0.3)
                )
            else:
                model = create_model(model_type)
            
            # Train with early stopping
            trainer = ModelTrainer(
                model, 
                device=device, 
                learning_rate=params.get('learning_rate', 0.001)
            )
            
            # Shorter training for optimization
            history = trainer.train(train_loader, val_loader, epochs=20)
            
            # Evaluate
            val_loss, val_acc = trainer.validate_epoch(val_loader)
            
            # Store results
            trial_results.append({
                'trial': trial + 1,
                'params': params.copy(),
                'val_acc': val_acc,
                'val_loss': val_loss,
                'final_epoch': len(history['train_loss'])
            })
            
            print(f"Validation Accuracy: {val_acc:.2f}%")
            
            # Update best
            if val_acc > best_score:
                best_score = val_acc
                best_params = params.copy()
                print(f"🎯 New best score: {best_score:.2f}%")
            
        except Exception as e:
            print(f"❌ Trial failed: {e}")
            trial_results.append({
                'trial': trial + 1,
                'params': params.copy(),
                'error': str(e)
            })
    
    # Display results
    print(f"\n{'='*60}")
    print(f"HYPERPARAMETER OPTIMIZATION RESULTS")
    print(f"{'='*60}")
    
    # Create results DataFrame
    successful_trials = [r for r in trial_results if 'error' not in r]
    
    if successful_trials:
        df = pd.DataFrame([
            {
                'Trial': r['trial'],
                'Accuracy': f"{r['val_acc']:.2f}%",
                'Loss': f"{r['val_loss']:.4f}",
                'LR': r['params'].get('learning_rate', 'N/A'),
                'Batch Size': r['params'].get('batch_size', 'N/A'),
                'Dropout': r['params'].get('dropout_rate', 'N/A'),
                'Hidden Size': r['params'].get('hidden_size', 'N/A'),
                'Layers': r['params'].get('num_layers', 'N/A')
            }
            for r in successful_trials
        ])
        
        print(df.to_string(index=False))
        
        # Plot optimization results
        plt.figure(figsize=(12, 8))
        
        # Plot accuracy vs trial
        plt.subplot(2, 2, 1)
        trials = [r['trial'] for r in successful_trials]
        accs = [r['val_acc'] for r in successful_trials]
        plt.plot(trials, accs, 'bo-')
        plt.title('Validation Accuracy vs Trial')
        plt.xlabel('Trial')
        plt.ylabel('Accuracy (%)')
        plt.grid(True)
        
        # Plot learning rate vs accuracy
        plt.subplot(2, 2, 2)
        lrs = [r['params']['learning_rate'] for r in successful_trials]
        plt.scatter(lrs, accs, alpha=0.7)
        plt.xscale('log')
        plt.title('Learning Rate vs Accuracy')
        plt.xlabel('Learning Rate')
        plt.ylabel('Accuracy (%)')
        plt.grid(True)
        
        # Plot batch size vs accuracy
        plt.subplot(2, 2, 3)
        batch_sizes = [r['params']['batch_size'] for r in successful_trials]
        plt.scatter(batch_sizes, accs, alpha=0.7)
        plt.title('Batch Size vs Accuracy')
        plt.xlabel('Batch Size')
        plt.ylabel('Accuracy (%)')
        plt.grid(True)
        
        # Plot dropout vs accuracy
        plt.subplot(2, 2, 4)
        dropouts = [r['params']['dropout_rate'] for r in successful_trials]
        plt.scatter(dropouts, accs, alpha=0.7)
        plt.title('Dropout Rate vs Accuracy')
        plt.xlabel('Dropout Rate')
        plt.ylabel('Accuracy (%)')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
    
    print(f"\n🏆 Best hyperparameters for {model_type.upper()}:")
    print(f"   Score: {best_score:.2f}%")
    print(f"   Parameters: {best_params}")
    
    return best_params, best_score, trial_results

def train_optimized_model(X, y, model_type, best_params):
    """Train model with optimized hyperparameters."""
    
    print(f"\n🚀 Training optimized {model_type.upper()} model...")
    
    # Prepare data
    train_loader, val_loader, test_loader = prepare_dataloaders(
        X, y, model_type, batch_size=best_params.get('batch_size', 32)
    )
    
    # Create optimized model
    if model_type == 'cnn':
        model = CNNModel(dropout_rate=best_params.get('dropout_rate', 0.3))
    elif model_type == 'lstm':
        model = LSTMModel(
            hidden_size=best_params.get('hidden_size', 128),
            num_layers=best_params.get('num_layers', 2),
            dropout_rate=best_params.get('dropout_rate', 0.3)
        )
    elif model_type == 'cnn_lstm':
        model = CNNLSTMModel(
            lstm_hidden_size=best_params.get('hidden_size', 128),
            lstm_num_layers=best_params.get('num_layers', 2),
            dropout_rate=best_params.get('dropout_rate', 0.3)
        )
    else:
        model = create_model(model_type)
    
    # Train with optimal parameters
    trainer = ModelTrainer(
        model, 
        device=device, 
        learning_rate=best_params.get('learning_rate', 0.001)
    )
    
    history = trainer.train(train_loader, val_loader, epochs=EPOCHS)
    
    # Final evaluation
    test_loss, test_acc = trainer.validate_epoch(test_loader)
    
    print(f"\n✅ Optimized {model_type.upper()} Results:")
    print(f"   Test Accuracy: {test_acc:.2f}%")
    print(f"   Test Loss: {test_loss:.4f}")
    
    return trainer.model, history, test_acc

# Run hyperparameter optimization (optional)
OPTIMIZE_HYPERPARAMETERS = False  # Set to True to run optimization

if OPTIMIZE_HYPERPARAMETERS and 'X' in locals() and 'y' in locals():
    print("🔬 Starting hyperparameter optimization...")
    
    # Choose model to optimize
    model_to_optimize = 'cnn'  # Change to 'lstm' or 'cnn_lstm' as needed
    
    # Run optimization
    best_params, best_score, optimization_results = optimize_hyperparameters(
        X, y, model_type=model_to_optimize, max_trials=10
    )
    
    # Train final optimized model
    if best_params:
        optimized_model, optimized_history, optimized_score = train_optimized_model(
            X, y, model_to_optimize, best_params
        )
        
        print(f"\n🎯 Optimization completed!")
        print(f"   Improvement: {optimized_score:.2f}% vs baseline")
    
else:
    print("⚠️ Hyperparameter optimization disabled")
    print("Set OPTIMIZE_HYPERPARAMETERS = True to enable")

In [ ]:
# Model Export and Deployment Preparation
def save_model(model, model_name, save_path="./models/"):
    """Save trained model for deployment."""
    
    import os
    
    # Create directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)
    
    # Save model
    model_path = os.path.join(save_path, f"{model_name}.pth")
    torch.save(model.state_dict(), model_path)
    
    # Save complete model (including architecture)
    complete_model_path = os.path.join(save_path, f"{model_name}_complete.pth")
    torch.save(model, complete_model_path)
    
    print(f"✅ Model saved:")
    print(f"   State dict: {model_path}")
    print(f"   Complete model: {complete_model_path}")
    
    return model_path, complete_model_path

def load_model(model_path, model_class=None):
    """Load saved model for inference."""
    
    try:
        if model_class is not None:
            # Load state dict into model class
            model = model_class()
            model.load_state_dict(torch.load(model_path, map_location=device))
        else:
            # Load complete model
            model = torch.load(model_path, map_location=device)
        
        model.eval()
        print(f"✅ Model loaded from {model_path}")
        return model
    
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None

def export_model_info(training_results, save_path="./models/"):
    """Export model information and metadata."""
    
    import json
    import os
    
    os.makedirs(save_path, exist_ok=True)
    
    # Prepare model info
    model_info = {
        'training_timestamp': pd.Timestamp.now().isoformat(),
        'models': {},
        'dataset_info': {
            'total_samples': len(X) if 'X' in locals() else 0,
            'input_shape': [64, 64, 3],
            'output_classes': 2,
            'class_names': ['No Press', 'Key Press']
        },
        'training_config': {
            'epochs': EPOCHS,
            'batch_size': BATCH_SIZE,
            'learning_rate': LEARNING_RATE,
            'device': str(device)
        }
    }
    
    # Add model-specific info
    for model_name, result in training_results.items():
        if 'error' not in result:
            model_info['models'][model_name] = {
                'test_accuracy': float(result['test_acc']),
                'test_loss': float(result['test_loss']),
                'training_epochs': len(result['history']['train_loss']),
                'best_val_accuracy': float(max(result['history']['val_acc'])),
                'best_val_loss': float(min(result['history']['val_loss'])),
                'model_size_mb': sum(p.numel() * p.element_size() for p in result['model'].parameters()) / 1024 / 1024
            }
    
    # Save info
    info_path = os.path.join(save_path, "model_info.json")
    with open(info_path, 'w') as f:
        json.dump(model_info, f, indent=2)
    
    print(f"✅ Model info saved to {info_path}")
    return model_info

def create_inference_script(best_model_name, save_path="./"):
    """Create a standalone inference script."""
    
    inference_script = f'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
from PIL import Image

# Model Architecture (copy from notebook)
class CNNModel(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class KeypressDetector:
    def __init__(self, model_path, device='cpu'):
        self.device = torch.device(device)
        self.model = CNNModel()
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()
        
    def preprocess_image(self, image):
        """Preprocess image for inference."""
        if isinstance(image, str):
            # Load from file
            image = Image.open(image).convert('RGB')
        
        # Resize to 64x64
        image = image.resize((64, 64))
        
        # Convert to array and normalize
        image_array = np.array(image).astype(np.float32) / 255.0
        
        # Add batch dimension and rearrange channels
        image_tensor = torch.FloatTensor(image_array).permute(2, 0, 1).unsqueeze(0)
        
        return image_tensor.to(self.device)
    
    def predict(self, image):
        """Predict keypress for an image."""
        with torch.no_grad():
            image_tensor = self.preprocess_image(image)
            output = self.model(image_tensor)
            probability = torch.sigmoid(output).cpu().numpy()[0][0]
            prediction = int(probability > 0.5)
            
        return prediction, probability

# Example usage
if __name__ == "__main__":
    # Initialize detector
    detector = KeypressDetector("models/{best_model_name}.pth")
    
    # Example prediction
    # prediction, probability = detector.predict("path/to/image.jpg")
    # print(f"Prediction: {{'Press' if prediction else 'No Press'}}, Probability: {{probability:.3f}}")
'''
    
    script_path = os.path.join(save_path, "keypress_inference.py")
    with open(script_path, 'w') as f:
        f.write(inference_script)
    
    print(f"✅ Inference script saved to {script_path}")
    return script_path

def create_deployment_requirements():
    """Create requirements.txt for deployment."""
    
    requirements = [
        "torch>=1.9.0",
        "torchvision>=0.10.0",
        "numpy>=1.21.0",
        "opencv-python>=4.5.0",
        "Pillow>=8.3.0",
        "scikit-learn>=1.0.0",
        "matplotlib>=3.4.0",
        "seaborn>=0.11.0",
        "pandas>=1.3.0",
        "tqdm>=4.62.0"
    ]
    
    with open("requirements.txt", 'w') as f:
        f.write("\\n".join(requirements))
    
    print("✅ Requirements.txt created")
    return requirements

# Export models and create deployment files
if 'training_results' in locals():
    print("📦 Preparing models for deployment...")
    
    # Find best model
    best_model_name = max(training_results.keys(), 
                         key=lambda x: training_results[x]['test_acc'] if 'error' not in training_results[x] else 0)
    
    print(f"🏆 Best model: {best_model_name.upper()}")
    
    # Save all models
    saved_models = {}
    for model_name, result in training_results.items():
        if 'error' not in result:
            model_path, complete_path = save_model(result['model'], model_name)
            saved_models[model_name] = {
                'state_dict_path': model_path,
                'complete_path': complete_path
            }
    
    # Export model information
    model_info = export_model_info(training_results)
    
    # Create inference script
    inference_script_path = create_inference_script(best_model_name)
    
    # Create requirements
    requirements = create_deployment_requirements()
    
    print(f"\\n🎉 Deployment preparation completed!")
    print(f"   Best model: {best_model_name.upper()} ({training_results[best_model_name]['test_acc']:.2f}%)")
    print(f"   Total models saved: {len(saved_models)}")
    print(f"   Inference script: keypress_inference.py")
    print(f"   Requirements: requirements.txt")
    print(f"   Model info: models/model_info.json")
    
    # Display final summary
    print(f"\\n{'='*60}")
    print(f"FINAL TRAINING SUMMARY")
    print(f"{'='*60}")
    
    for model_name, result in training_results.items():
        if 'error' not in result:
            print(f"{model_name.upper():12} - Accuracy: {result['test_acc']:5.2f}% | Loss: {result['test_loss']:.4f}")
    
    print(f"\\n🚀 Ready for deployment!")
    
else:
    print("❌ No training results available for export")
    print("Please run the training cell first")

# 🎯 Training Complete - Summary & Next Steps

## 📊 What We've Accomplished

This comprehensive notebook provides a complete multi-model training pipeline for keypress detection:

### 🤖 **Multiple Model Architectures**
- **CNN Model**: Convolutional Neural Network for image classification
- **LSTM Model**: Long Short-Term Memory for sequence processing
- **CNN+LSTM Model**: Combined approach for spatial-temporal features
- **ResNet Model**: Deep residual network for robust feature extraction
- **Transformer Model**: Attention-based architecture for advanced pattern recognition

### 🔧 **Training Infrastructure**
- **ModelTrainer Class**: Unified training loop with early stopping
- **Custom Dataset**: PyTorch dataset for keypress data
- **Data Augmentation**: Random transforms for better generalization
- **Cross-validation**: Robust model evaluation
- **Hyperparameter Optimization**: Automatic tuning for best performance

### 📈 **Analysis & Evaluation**
- **Comprehensive Metrics**: Accuracy, Precision, Recall, F1-Score, AUC
- **Visualization**: Training curves, confusion matrices, prediction samples
- **Model Comparison**: Side-by-side performance analysis
- **Speed Testing**: Inference performance evaluation

### 🚀 **Deployment Ready**
- **Model Export**: Save trained models for production
- **Inference Script**: Standalone prediction code
- **Requirements**: Complete dependency list
- **Documentation**: Model metadata and configuration

## 🎮 **Integration with Video Labeler**

This notebook perfectly complements your `video_labeler.py` tool:

1. **Data Flow**: Video Labeler → JSON Training Data → This Notebook → Trained Models
2. **Format Compatibility**: Direct support for video labeler output format
3. **Real-time Integration**: Trained models can be used in real-time detection

## 🔄 **Recommended Workflow**

1. **Label Data**: Use `video_labeler.py` to create training dataset
2. **Train Models**: Run this notebook to train multiple architectures
3. **Evaluate**: Compare model performance and select best one
4. **Deploy**: Use exported models in your keypress detection system
5. **Iterate**: Collect more data and retrain for better performance

## 🎯 **Next Steps**

### **Immediate Actions**
- [ ] Run the notebook with your labeled data
- [ ] Compare model performance and select best architecture
- [ ] Export the best model for deployment
- [ ] Test inference speed on your target hardware

### **Advanced Improvements**
- [ ] Collect more diverse training data
- [ ] Experiment with data augmentation techniques
- [ ] Try ensemble methods combining multiple models
- [ ] Implement online learning for continuous improvement

### **Production Deployment**
- [ ] Integrate best model into your detection pipeline
- [ ] Set up model versioning and monitoring
- [ ] Create automated retraining pipeline
- [ ] Add performance metrics tracking

## 🤝 **Support & Resources**

- **Model Files**: All trained models saved in `./models/` directory
- **Inference Script**: `keypress_inference.py` for standalone predictions
- **Documentation**: `model_info.json` contains all training metadata
- **Requirements**: `requirements.txt` for deployment environment

## 🎉 **Ready to Deploy!**

Your keypress detection system is now ready for production use. The trained models can accurately detect key presses in real-time video streams, providing the foundation for your hand gesture recognition application.

---

*Happy Training! 🚀*

In [ ]:
# Train the Model
def create_callbacks(model_name="keypress_model"):
    """Create callbacks for training."""
    
    # Create model directory
    model_dir = f"models/{model_name}"
    os.makedirs(model_dir, exist_ok=True)
    
    callbacks = [
        # Model checkpoint - save best model
        keras.callbacks.ModelCheckpoint(
            filepath=f"{model_dir}/best_model.h5",
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            verbose=1
        ),
        
        # Early stopping
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Learning rate reduction
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-7,
            verbose=1
        ),
        
        # CSV logger
        keras.callbacks.CSVLogger(
            f"{model_dir}/training_log.csv",
            append=True
        ),
        
        # TensorBoard
        keras.callbacks.TensorBoard(
            log_dir=f"{model_dir}/tensorboard_logs",
            histogram_freq=1,
            write_graph=True,
            write_images=True
        )
    ]
    
    return callbacks

def train_model(model, train_data, val_data, epochs=100, model_name="keypress_model"):
    """Train the model with callbacks."""
    
    print(f"Starting training for {epochs} epochs...")
    print(f"Model will be saved to: models/{model_name}/")
    
    # Create callbacks
    callbacks = create_callbacks(model_name)
    
    # Train the model
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    return history

def plot_training_history(history):
    """Plot training history."""
    
    if not history:
        print("No training history available")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot training & validation accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Plot training & validation loss
    axes[0, 1].plot(history.history['loss'], label='Training Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Plot precision
    if 'precision' in history.history:
        axes[1, 0].plot(history.history['precision'], label='Training Precision')
        axes[1, 0].plot(history.history['val_precision'], label='Validation Precision')
        axes[1, 0].set_title('Model Precision')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Precision')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    # Plot recall
    if 'recall' in history.history:
        axes[1, 1].plot(history.history['recall'], label='Training Recall')
        axes[1, 1].plot(history.history['val_recall'], label='Validation Recall')
        axes[1, 1].set_title('Model Recall')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Recall')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

# Train the model
if 'model' in locals() and 'data_splits' in locals():
    print("Starting model training...")
    
    # Create model directory
    os.makedirs("models", exist_ok=True)
    
    # Train the model
    history = train_model(
        model=model,
        train_data=train_gen,
        val_data=val_gen,
        epochs=50,  # Start with 50 epochs
        model_name="cnn_lstm_keypress"
    )
    
    # Plot training history
    plot_training_history(history)
    
    print("Training completed!")
    
else:
    print("Model or data not available for training")
    print("Please ensure you have:")
    print("1. Loaded and preprocessed training data")
    print("2. Created the model")
    print("3. Prepared data splits")

In [ ]:
# Evaluate Model Performance
def evaluate_model(model, X_test, y_test):
    """Evaluate model on test data."""
    
    print("Evaluating model on test data...")
    
    # Make predictions
    predictions = model.predict(X_test, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Test Results:")
    print(f"  - Accuracy: {accuracy:.4f}")
    print(f"  - Precision: {precision:.4f}")
    print(f"  - Recall: {recall:.4f}")
    print(f"  - F1-Score: {f1:.4f}")
    
    return y_pred, y_true, predictions

def plot_confusion_matrix(y_true, y_pred, class_names=['Not Pressed', 'Pressed']):
    """Plot confusion matrix."""
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # Print detailed metrics
    print("\nDetailed Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

def plot_prediction_confidence(predictions, y_true, num_samples=100):
    """Plot prediction confidence distribution."""
    
    # Select random samples
    indices = np.random.choice(len(predictions), min(num_samples, len(predictions)), replace=False)
    
    confidence_scores = np.max(predictions[indices], axis=1)
    true_labels = y_true[indices]
    
    # Separate by correct/incorrect predictions
    pred_labels = np.argmax(predictions[indices], axis=1)
    correct_mask = (pred_labels == true_labels)
    
    plt.figure(figsize=(12, 4))
    
    # Plot confidence for correct predictions
    plt.subplot(1, 2, 1)
    plt.hist(confidence_scores[correct_mask], bins=20, alpha=0.7, 
             color='green', label='Correct Predictions')
    plt.hist(confidence_scores[~correct_mask], bins=20, alpha=0.7, 
             color='red', label='Incorrect Predictions')
    plt.xlabel('Prediction Confidence')
    plt.ylabel('Frequency')
    plt.title('Prediction Confidence Distribution')
    plt.legend()
    
    # Plot confidence by class
    plt.subplot(1, 2, 2)
    for class_idx in range(2):
        class_mask = (true_labels == class_idx)
        plt.hist(confidence_scores[class_mask], bins=20, alpha=0.7, 
                 label=f'Class {class_idx}')
    plt.xlabel('Prediction Confidence')
    plt.ylabel('Frequency')
    plt.title('Confidence by True Class')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def analyze_errors(X_test, y_true, y_pred, predictions, num_errors=8):
    """Analyze prediction errors."""
    
    # Find incorrect predictions
    incorrect_indices = np.where(y_true != y_pred)[0]
    
    if len(incorrect_indices) == 0:
        print("No prediction errors found!")
        return
    
    print(f"Found {len(incorrect_indices)} prediction errors out of {len(y_true)} samples")
    print(f"Error rate: {len(incorrect_indices)/len(y_true):.2%}")
    
    # Show some error cases
    num_to_show = min(num_errors, len(incorrect_indices))
    error_indices = np.random.choice(incorrect_indices, num_to_show, replace=False)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('Prediction Errors Analysis', fontsize=16)
    
    for i, idx in enumerate(error_indices):
        row = i // 4
        col = i % 4
        
        axes[row, col].imshow(X_test[idx])
        
        true_label = "Pressed" if y_true[idx] == 1 else "Not Pressed"
        pred_label = "Pressed" if y_pred[idx] == 1 else "Not Pressed"
        confidence = predictions[idx][y_pred[idx]]
        
        axes[row, col].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.3f}')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Evaluate the model
if 'model' in locals() and 'data_splits' in locals():
    # Load best model if available
    try:
        best_model = keras.models.load_model("models/cnn_lstm_keypress/best_model.h5")
        print("Loaded best model from checkpoint")
        model = best_model
    except:
        print("Using current model (no checkpoint found)")
    
    # Evaluate on test data
    y_pred, y_true, predictions = evaluate_model(model, data_splits['X_test'], data_splits['y_test'])
    
    # Plot confusion matrix
    plot_confusion_matrix(y_true, y_pred)
    
    # Plot prediction confidence
    plot_prediction_confidence(predictions, y_true)
    
    # Analyze errors
    analyze_errors(data_splits['X_test'], y_true, y_pred, predictions)
    
else:
    print("Model or data not available for evaluation")
    print("Please ensure you have trained the model first")

In [ ]:
# Real-time Inference Testing
def create_inference_pipeline(model, target_key='t', model_path='best_v2.pt'):
    """Create inference pipeline for real-time testing."""
    
    from ultralytics import YOLO
    
    # Load YOLO model for key detection
    yolo_model = YOLO(model_path)
    
    def predict_keypress(frame, key_bbox=None):
        """Predict keypress from a single frame."""
        
        if key_bbox is None:
            # Detect key in frame
            results = yolo_model(frame)
            
            for result in results:
                if result.boxes is not None:
                    for box in result.boxes:
                        if result.names[int(box.cls)] == target_key:
                            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                            key_bbox = (int(x1), int(y1), int(x2), int(y2))
                            break
            
            if key_bbox is None:
                return None, None  # Key not found
        
        # Extract and preprocess key region
        x1, y1, x2, y2 = key_bbox
        
        # Expand bounding box by 20% (same as video labeler)
        width = x2 - x1
        height = y2 - y1
        expand_x = width * 0.2
        expand_y = height * 0.2
        
        exp_x1 = max(0, int(x1 - expand_x))
        exp_y1 = max(0, int(y1 - expand_y))
        exp_x2 = min(frame.shape[1], int(x2 + expand_x))
        exp_y2 = min(frame.shape[0], int(y2 + expand_y))
        
        # Extract key region
        key_region = frame[exp_y1:exp_y2, exp_x1:exp_x2]
        
        # Resize to model input size
        key_region = cv2.resize(key_region, (64, 64))
        
        # Normalize
        key_region = key_region.astype(np.float32) / 255.0
        
        # Add batch dimension
        key_region = np.expand_dims(key_region, axis=0)
        
        # Make prediction
        prediction = model.predict(key_region, verbose=0)
        
        # Get probability and class
        prob = prediction[0]
        predicted_class = np.argmax(prob)
        confidence = prob[predicted_class]
        
        return predicted_class, confidence
    
    return predict_keypress

def test_on_video(video_path, model, target_key='t', output_path=None):
    """Test model on a video file."""
    
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video: {video_path}")
    print(f"FPS: {fps}, Size: {width}x{height}, Frames: {total_frames}")
    
    # Create inference pipeline
    predict_keypress = create_inference_pipeline(model, target_key)
    
    # Setup video writer if output path is provided
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Process video
    frame_count = 0
    key_bbox = None
    predictions = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Predict keypress
        predicted_class, confidence = predict_keypress(frame, key_bbox)
        
        if predicted_class is not None:
            predictions.append({
                'frame': frame_count,
                'prediction': predicted_class,
                'confidence': confidence
            })
            
            # Draw prediction on frame
            label = "PRESSED" if predicted_class == 1 else "NOT PRESSED"
            color = (0, 255, 0) if predicted_class == 1 else (0, 0, 255)
            
            cv2.putText(frame, f"{label} ({confidence:.3f})", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            
            # Draw key bounding box if found
            if key_bbox:
                x1, y1, x2, y2 = key_bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # Write frame to output video
        if output_path:
            out.write(frame)
        
        # Display frame (optional)
        cv2.imshow('Keypress Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()
    
    print(f"Processed {frame_count} frames")
    print(f"Made {len(predictions)} predictions")
    
    return predictions

def analyze_video_predictions(predictions):
    """Analyze predictions from video inference."""
    
    if not predictions:
        print("No predictions to analyze")
        return
    
    # Convert to DataFrame for easier analysis
    df = pd.DataFrame(predictions)
    
    # Basic statistics
    print(f"Video Prediction Analysis:")
    print(f"  - Total frames predicted: {len(df)}")
    print(f"  - Pressed frames: {sum(df['prediction'] == 1)}")
    print(f"  - Not pressed frames: {sum(df['prediction'] == 0)}")
    print(f"  - Average confidence: {df['confidence'].mean():.3f}")
    
    # Plot predictions over time
    plt.figure(figsize=(15, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(df['frame'], df['prediction'], 'b-', linewidth=2)
    plt.title('Key Press Predictions Over Time')
    plt.ylabel('Prediction (0=Not Pressed, 1=Pressed)')
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(df['frame'], df['confidence'], 'r-', linewidth=2)
    plt.title('Prediction Confidence Over Time')
    plt.xlabel('Frame Number')
    plt.ylabel('Confidence')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Test the model on video
if 'model' in locals():
    # Example usage (uncomment and modify paths as needed)
    
    # Test on a video file
    # video_path = "path/to/your/test_video.mp4"
    # output_path = "keypress_predictions.mp4"
    
    # predictions = test_on_video(video_path, model, target_key='t', output_path=output_path)
    # analyze_video_predictions(predictions)
    
    print("Real-time inference pipeline created!")
    print("To test on a video:")
    print("1. Uncomment the lines above")
    print("2. Set video_path to your test video")
    print("3. Set output_path for result video")
    print("4. Run the cell")
    
else:
    print("Model not available for inference testing")
    print("Please train the model first")

In [ ]:
# Model Optimization and Fine-tuning
def create_optimized_model(input_shape=(64, 64, 3), num_classes=2):
    """Create optimized CNN model with better architecture."""
    
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    
    # Residual block 1
    residual = x
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Residual block 2
    residual = layers.Conv2D(64, (1, 1), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Residual block 3
    residual = layers.Conv2D(128, (1, 1), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Attention mechanism
    attention = layers.Conv2D(128, (1, 1), activation='sigmoid')(x)
    x = layers.Multiply()([x, attention])
    
    # Global pooling and dense layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def fine_tune_model(base_model, X_train, y_train, X_val, y_val, epochs=20):
    """Fine-tune model with different learning rates."""
    
    print("Fine-tuning model...")
    
    # Freeze some layers
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    # Compile with lower learning rate
    base_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Create callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7
        )
    ]
    
    # Fine-tune
    history = base_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    return history

def hyperparameter_tuning(X_train, y_train, X_val, y_val):
    """Perform hyperparameter tuning."""
    
    print("Performing hyperparameter tuning...")
    
    # Define hyperparameter search space
    lr_values = [0.001, 0.0005, 0.0001]
    batch_sizes = [16, 32, 64]
    dropout_rates = [0.3, 0.5, 0.7]
    
    best_accuracy = 0
    best_params = {}
    results = []
    
    for lr in lr_values:
        for batch_size in batch_sizes:
            for dropout in dropout_rates:
                print(f"Testing: LR={lr}, Batch={batch_size}, Dropout={dropout}")
                
                # Create model with current hyperparameters
                model = create_optimized_model()
                
                # Modify dropout rate
                for layer in model.layers:
                    if isinstance(layer, layers.Dropout):
                        layer.rate = dropout
                
                # Compile model
                model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )
                
                # Train for a few epochs
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=batch_size,
                    verbose=0
                )
                
                # Get best validation accuracy
                val_accuracy = max(history.history['val_accuracy'])
                
                results.append({
                    'lr': lr,
                    'batch_size': batch_size,
                    'dropout': dropout,
                    'val_accuracy': val_accuracy
                })
                
                if val_accuracy > best_accuracy:
                    best_accuracy = val_accuracy
                    best_params = {
                        'lr': lr,
                        'batch_size': batch_size,
                        'dropout': dropout
                    }
                
                print(f"  Validation accuracy: {val_accuracy:.4f}")
    
    print(f"\nBest parameters: {best_params}")
    print(f"Best validation accuracy: {best_accuracy:.4f}")
    
    return best_params, results

def ensemble_prediction(models, X_test):
    """Create ensemble predictions from multiple models."""
    
    predictions = []
    
    for model in models:
        pred = model.predict(X_test, verbose=0)
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    
    return ensemble_pred

# Model optimization example
if 'data_splits' in locals():
    print("Starting model optimization...")
    
    # Create optimized model
    optimized_model = create_optimized_model()
    
    print("Optimized Model Architecture:")
    optimized_model.summary()
    
    # Compile optimized model
    optimized_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Optional: Perform hyperparameter tuning (this will take a while)
    # best_params, tuning_results = hyperparameter_tuning(
    #     data_splits['X_train'], data_splits['y_train'],
    #     data_splits['X_val'], data_splits['y_val']
    # )
    
    print("Model optimization setup completed!")
    print("Uncomment hyperparameter tuning section to run full optimization")
    
else:
    print("Data not available for model optimization")
    print("Please prepare training data first")

In [ ]:
# Export Model for Production
def export_model(model, model_name="keypress_detector"):
    """Export trained model in multiple formats for production."""
    
    # Create export directory
    export_dir = f"exported_models/{model_name}"
    os.makedirs(export_dir, exist_ok=True)
    
    print(f"Exporting model to {export_dir}/...")
    
    # 1. Save in Keras H5 format
    h5_path = f"{export_dir}/{model_name}.h5"
    model.save(h5_path)
    print(f"✓ Saved H5 model: {h5_path}")
    
    # 2. Save in TensorFlow SavedModel format
    savedmodel_path = f"{export_dir}/{model_name}_savedmodel"
    model.save(savedmodel_path)
    print(f"✓ Saved TensorFlow SavedModel: {savedmodel_path}")
    
    # 3. Save model weights only
    weights_path = f"{export_dir}/{model_name}_weights.h5"
    model.save_weights(weights_path)
    print(f"✓ Saved model weights: {weights_path}")
    
    # 4. Export to TensorFlow Lite for mobile deployment
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        
        tflite_path = f"{export_dir}/{model_name}.tflite"
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"✓ Saved TensorFlow Lite model: {tflite_path}")
        
        # Get model size
        model_size = os.path.getsize(tflite_path) / 1024  # KB
        print(f"  TensorFlow Lite model size: {model_size:.1f} KB")
        
    except Exception as e:
        print(f"✗ TensorFlow Lite export failed: {e}")
    
    # 5. Save model architecture as JSON
    architecture_path = f"{export_dir}/{model_name}_architecture.json"
    with open(architecture_path, 'w') as f:
        f.write(model.to_json())
    print(f"✓ Saved model architecture: {architecture_path}")
    
    # 6. Save model summary
    summary_path = f"{export_dir}/{model_name}_summary.txt"
    with open(summary_path, 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"✓ Saved model summary: {summary_path}")
    
    return export_dir

def create_inference_class(model_path, target_key='t'):
    """Create a production-ready inference class."""
    
    inference_code = f'''
import tensorflow as tf
import numpy as np
import cv2
from ultralytics import YOLO

class KeypressDetector:
    def __init__(self, model_path="{model_path}", yolo_model_path="best_v2.pt"):
        """Initialize the keypress detector."""
        self.model = tf.keras.models.load_model(model_path)
        self.yolo_model = YOLO(yolo_model_path)
        self.target_key = "{target_key}"
        self.key_bbox = None
        
    def detect_key_region(self, frame):
        """Detect key region in frame using YOLO."""
        results = self.yolo_model(frame)
        
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    if result.names[int(box.cls)] == self.target_key:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        
                        # Expand bounding box by 20%
                        width = x2 - x1
                        height = y2 - y1
                        expand_x = width * 0.2
                        expand_y = height * 0.2
                        
                        exp_x1 = max(0, int(x1 - expand_x))
                        exp_y1 = max(0, int(y1 - expand_y))
                        exp_x2 = min(frame.shape[1], int(x2 + expand_x))
                        exp_y2 = min(frame.shape[0], int(y2 + expand_y))
                        
                        self.key_bbox = (exp_x1, exp_y1, exp_x2, exp_y2)
                        return self.key_bbox
        
        return None
    
    def preprocess_frame(self, frame, bbox=None):
        """Preprocess frame for model input."""
        if bbox is None:
            bbox = self.key_bbox
        
        if bbox is None:
            return None
        
        # Extract key region
        x1, y1, x2, y2 = bbox
        key_region = frame[y1:y2, x1:x2]
        
        # Resize to model input size
        key_region = cv2.resize(key_region, (64, 64))
        
        # Normalize
        key_region = key_region.astype(np.float32) / 255.0
        
        # Add batch dimension
        key_region = np.expand_dims(key_region, axis=0)
        
        return key_region
    
    def predict(self, frame, bbox=None):
        """Predict keypress from frame."""
        # Preprocess frame
        input_data = self.preprocess_frame(frame, bbox)
        
        if input_data is None:
            return None, None
        
        # Make prediction
        prediction = self.model.predict(input_data, verbose=0)
        
        # Get result
        prob = prediction[0]
        predicted_class = np.argmax(prob)
        confidence = prob[predicted_class]
        
        return predicted_class, confidence
    
    def process_video(self, video_path, output_path=None):
        """Process entire video and return predictions."""
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            raise ValueError(f"Cannot open video: {{video_path}}")
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Setup video writer if output path provided
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        predictions = []
        frame_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Detect key region on first frame
            if self.key_bbox is None:
                self.detect_key_region(frame)
            
            # Make prediction
            predicted_class, confidence = self.predict(frame)
            
            if predicted_class is not None:
                predictions.append({{
                    'frame': frame_count,
                    'prediction': predicted_class,
                    'confidence': confidence
                }})
                
                # Draw prediction on frame
                label = "PRESSED" if predicted_class == 1 else "NOT PRESSED"
                color = (0, 255, 0) if predicted_class == 1 else (0, 0, 255)
                
                cv2.putText(frame, f"{{label}} ({{confidence:.3f}})", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                
                # Draw bounding box
                if self.key_bbox:
                    x1, y1, x2, y2 = self.key_bbox
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Write frame to output
            if output_path:
                out.write(frame)
        
        # Cleanup
        cap.release()
        if output_path:
            out.release()
        
        return predictions

# Example usage:
# detector = KeypressDetector()
# predictions = detector.process_video("input_video.mp4", "output_video.mp4")
'''
    
    return inference_code

def save_inference_code(code, export_dir):
    """Save inference code to file."""
    code_path = f"{export_dir}/keypress_detector.py"
    with open(code_path, 'w') as f:
        f.write(code)
    print(f"✓ Saved inference code: {code_path}")
    return code_path

def create_deployment_requirements():
    """Create requirements.txt for deployment."""
    requirements = '''
tensorflow>=2.8.0
opencv-python>=4.5.0
ultralytics>=8.0.0
numpy>=1.20.0
'''
    return requirements

def save_deployment_files(export_dir):
    """Save deployment files."""
    
    # Save requirements.txt
    requirements_path = f"{export_dir}/requirements.txt"
    with open(requirements_path, 'w') as f:
        f.write(create_deployment_requirements())
    print(f"✓ Saved requirements: {requirements_path}")
    
    # Save README
    readme_content = f'''
# Keypress Detection Model

This directory contains the trained keypress detection model and inference code.

## Files:
- `keypress_detector.h5`: Trained Keras model
- `keypress_detector_savedmodel/`: TensorFlow SavedModel format
- `keypress_detector.tflite`: TensorFlow Lite model for mobile
- `keypress_detector.py`: Inference class for production use
- `requirements.txt`: Required Python packages

## Usage:
```python
from keypress_detector import KeypressDetector

# Initialize detector
detector = KeypressDetector("keypress_detector.h5")

# Process video
predictions = detector.process_video("input.mp4", "output.mp4")
```

## Installation:
```bash
pip install -r requirements.txt
```

Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
'''
    
    readme_path = f"{export_dir}/README.md"
    with open(readme_path, 'w') as f:
        f.write(readme_content)
    print(f"✓ Saved README: {readme_path}")

# Export the model for production
if 'model' in locals():
    # Export model
    export_dir = export_model(model, "keypress_detector")
    
    # Create inference code
    model_path = f"{export_dir}/keypress_detector.h5"
    inference_code = create_inference_class(model_path)
    save_inference_code(inference_code, export_dir)
    
    # Save deployment files
    save_deployment_files(export_dir)
    
    print(f"\n🎉 Model successfully exported to: {export_dir}")
    print("\nProduction-ready files created:")
    print("- Model files (H5, SavedModel, TensorFlow Lite)")
    print("- Inference code (keypress_detector.py)")
    print("- Requirements and documentation")
    print("\nYour model is ready for deployment!")
    
else:
    print("Model not available for export")
    print("Please train the model first")